In [1]:
import rl
import utils
import agents
import torch.nn as nn
from time import sleep
from stable_baselines3.common.torch_layers import NatureCNN

In [2]:
vec_env = rl.setup_vec_env(
        n_envs=8,
        frame_skip=4,
        action_repeat_probability=0.0,
        frame_stacks=4,
        seed=0,
)

callback = rl.setup_callback(vec_env, n_envs=8, eval_freq=100_000, n_eval_episodes=5)

In [3]:
name="impala_cnn_1400k_v2"
model = rl.load(path=f"C:/Users/cgoet/PycharmProjects/Pong-RL/models/{name}", vec_env=vec_env)

In [3]:
model = rl.setup_model(
        vec_env=vec_env,
        model_class=agents.ImpalaCNN,
        model_kwargs={
            "depths": [16, 32, 64, 64, 128],
            "features_dim": 512,
            "activation_layer": nn.Mish,
        },
        net_arch={"pi": [64], "vf": [64]},
        device="cuda",
        log_dir="C:/Users/cgoet/PycharmProjects/Pong-RL/logs/",
)

In [25]:
model = rl.setup_model(
        vec_env=vec_env,
        model_class=NatureCNN,
        model_kwargs={"features_dim": 256},
        net_arch={"pi": [64], "vf": [64]},
        device="cuda",
        log_dir="C:/Users/cgoet/PycharmProjects/Pong-RL/logs/",
)

In [4]:
utils.print_model_parameters(model, shared_extractor=True)

features_extractor: 1,329,200
pi_features_extractor: 1,329,200
vf_features_extractor: 1,329,200
mlp_extractor: 65,664
action_net: 390
value_net: 65
Total number of parameters: 1,395,319


In [ ]:
# tensorboard --logdir="C:/Users/cgoet/PycharmProjects/Pong-RL/logs"

In [5]:
name="impala_cnn_1400k_v2"
model = rl.train(model, steps=1_000_000, name=name, new_run=True, callbacks=callback)  # start new training run

C:\Users\cgoet\PycharmProjects\Pong-RL\.venv\lib\site-packages\stable_baselines3\common\callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x000001F3EDF61120> != <stable_baselines3.common.vec_env.vec_frame_stack.VecFrameStack object at 0x000001F3EDCBE5F0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


In [20]:
model = rl.train(model, steps=100_000, name=name, new_run=False, callbacks=callback)  # continue training run

Eval num_timesteps=4103232, episode_reward=3.80 +/- 14.16
Episode length: 8825.00 +/- 2155.67


In [21]:
rl.save(model, path=f"C:/Users/cgoet/PycharmProjects/Pong-RL/models/{name}")

In [11]:
mean_reward, std_reward = rl.evaluate(model, vec_env, episodes=10, deterministic=True)

Mean reward: -17.10 +/- 3.83


In [12]:
mean_reward, std_reward = rl.evaluate(model, vec_env, episodes=10, deterministic=False)

Mean reward: -13.30 +/- 2.83


In [22]:
# test model fast
test_env = rl.setup_vec_env(
        n_envs=1,
        frame_skip=4,
        action_repeat_probability=0.0,
        frame_stacks=4,
        seed=0,
)

delay = 1 / 24
episodes = 1

episode_rewards = []
for episode in range(episodes):
    # obs shape (n_envs, 40, 40, n_frame_stacks)
    obs = test_env.reset()
    dones = False
    total_reward = 0

    while not dones:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, dones, info = test_env.step(action)
        total_reward += reward[0]
        test_env.render("human")
        sleep(delay)
        
    episode_rewards.append(total_reward)
    print(f"Episode {episode} reward: {total_reward}")

C:\Users\cgoet\PycharmProjects\Pong-RL\.venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 0 reward: 21.0


# Not working correctly

In [14]:
rewards = rl.visualize(model=None, screen_size=screen_size, episodes=1)  # random agent

Episode 0 reward: -20.0
Average reward: -20.0


In [13]:
rewards = rl.visualize(model, screen_size=screen_size, episodes=1)  # trained agent

Episode 0 reward: -21.0
Average reward: -21.0


In [10]:
env = gym.make(
    "ALE/Pong-v5",
    difficulty=None,
    obs_type="rgb",
    frameskip=4,
    repeat_action_probability=0.,
    render_mode="human",
)
input_env = WarpFrame(env, width=screen_size, height=screen_size)
input_env = DummyVecEnv([lambda: input_env])
input_env = VecFrameStack(input_env, n_stack=4)

episode_rewards = []
for episode in range(1):
    obs = input_env.reset()
    dones = False
    total_reward = 0

    while not dones:
        obs = np.moveaxis(obs, 3, 1)
        action, _ = model.predict(obs, deterministic=False)
        print(action)
        obs, reward, dones, info = input_env.step(action)
        total_reward += reward[0]
        break

    episode_rewards.append(total_reward)
    print(f"Episode {episode} reward: {total_reward}")

print(f"Average reward: {sum(episode_rewards) / len(episode_rewards)}")

episode_rewards

[5]
Episode 0 reward: 0.0
Average reward: 0.0


[0.0]